In [2]:
import sqlite3
import pandas as pd
import glob

In [10]:
# import data

data_dir = '/home/travis/data/s10_database'

data_list = glob.glob(os.path.join(data_dir, '*'))
data_list


# scanner3d_data = pd.read_csv('/home/travis/data/s10_database/2020-01-21_3d_volumes_entropy_vs009_fullres_newo3d.csv')
# rgb_data = pd.read_csv('/home/travis/data/s10_database/stereoTop_full_season_clustering.csv')
flir_data = pd.read_csv('/home/travis/data/database/full_season_named_flir_identifications_v5.csv')

In [6]:
# concatonating psii data
all_psii = glob.glob(os.path.join( '/home/travis/data/s10_database/fluorescence_outs', '*', '*'))

['/home/travis/data/s10_database/fluorescence_outs/2020-02-13_fluorescence_aggregation/2020-02-13_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-27_fluorescence_aggregation/2020-02-27_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-07_fluorescence_aggregation/2020-02-07_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-19_fluorescence_aggregation/2020-02-19_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-01-24_fluorescence_aggregation/2020-01-24_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-20_fluorescence_aggregation/2020-02-20_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-12_fluorescence_aggregation/2020-02-12_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-14_fluorescence_aggregation/2020-02-14

In [10]:
psii_data = pd.DataFrame()


for i in all_psii:
    df = pd.read_csv(i)
    psii_data = psii_data.append(df, ignore_index = True)



In [18]:
# get all columns for keys

print('psii:', psii_data.columns)
print('rgb:', rgb_data.columns)
print('flir:', flir_data.columns)
print('3d:', scanner3d_data.columns)

psii: Index(['Unnamed: 0', 'Plot', 'F0', 'FM', 'FV', 'FV/FM'], dtype='object')
rgb: Index(['Unnamed: 0', 'plant_name', 'date', 'treatment', 'plot', 'genotype',
       'lon', 'lat', 'min_x', 'max_x', 'min_y', 'max_y', 'nw_lat', 'nw_lon',
       'se_lat', 'se_lon', 'bounding_area_m2', 'double_lettuce',
       'Unnamed: 0.1', 'id', 'geometry', 'index_right', 'ID'],
      dtype='object')
flir: Index(['index', 'date', 'treatment', 'plot', 'genotype', 'lon', 'lat', 'min_x',
       'max_x', 'min_y', 'max_y', 'nw_lat', 'nw_lon', 'se_lat', 'se_lon',
       'bounding_area_m2', 'roi_temp', 'quartile_1', 'mean', 'median',
       'quartile_3', 'variance', 'std_dev', 'labels', 'plant_name',
       'double_lettuce'],
      dtype='object')
3d: Index(['plant_name', 'min_x', 'min_y', 'min_z', 'max_x', 'max_y', 'max_z',
       'num_points', 'hull_volume', 'oriented_bounding_box',
       'axis_aligned_bounding_box', 'persistence_entropy_0',
       'persistence_entropy_1', 'persistence_entropy_2', 'number_

In [23]:
# rgb needs cleaning

rgb_data = rgb_data.drop('id', 1)

In [26]:
# add rgb to database

with sqlite3.connect('po.db') as con:
    rgb_data.to_sql('rgb', con=con, dtype={('plant_name', 'date'): 'STRING PRIMARY KEY'})

/home/travis/Downloads/yes/envs/general/lib/python3.9/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [13]:
# Flir needs cleaning
flir = flir_data
is_dup_df = flir.duplicated(subset=['plant_name', 'date'],keep=False)
dups = flir[is_dup_df]
display(dups)


print(dups.plant_name.unique())

dups.to_csv('nan_plants_flir_season10_v5.csv')

,index,date,treatment,plot,genotype,lon,lat,min_x,max_x,min_y,...,roi_temp,quartile_1,mean,median,quartile_3,variance,std_dev,labels,plant_name,double_lettuce
7,8293,2020-02-11,treatment 1,MAC_Field_Scanner_Season_10_Range_12_Column_25,Parade,-111.974920,33.074941,91,162,661,...,293.280426,292.125046,293.470734,293.403290,294.783234,3.132295,1.769829,12,NaN,0
20,10804,2020-02-17,treatment 1,MAC_Field_Scanner_Season_10_Range_12_Column_25,Parade,-111.974920,33.074941,83,164,667,...,296.548950,295.450592,297.010986,297.126373,298.817963,5.982282,2.445870,12,NaN,0
24,4654,2020-02-26,treatment 1,MAC_Field_Scanner_Season_10_Range_12_Column_25,Parade,-111.974920,33.074940,77,161,693,...,294.212036,293.507812,294.753448,294.452057,295.779358,4.834380,2.198722,12,NaN,0
30,11362,2020-02-03,treatment 1,MAC_Field_Scanner_Season_10_Range_12_Column_25,Parade,-111.974920,33.074941,98,160,637,...,290.887756,290.585602,291.256317,291.076569,291.913391,0.914853,0.956480,12,NaN,0
34,5574,2020-02-12,treatment 1,MAC_Field_Scanner_Season_10_Range_12_Column_25,Parade,-111.974920,33.074942,75,153,552,...,292.346222,292.078156,293.146820,292.841187,293.769028,2.251432,1.500477,12,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200315,1147,2020-02-08,treatment 1,MAC_Field_Scanner_Season_10_Range_3_Column_6,Green_Towers,-111.975027,33.074614,0,68,720,...,296.428406,295.376465,296.520844,296.296722,297.463318,2.272268,1.507404,10,NaN,0
200317,1149,2020-02-08,treatment 1,MAC_Field_Scanner_Season_10_Range_3_Column_6,Green_Towers,-111.975024,33.074596,125,170,1546,...,295.795135,295.583679,296.570374,296.702118,297.598022,2.094549,1.447256,22,NaN,0
200417,4132,2020-02-12,treatment 2,MAC_Field_Scanner_Season_10_Range_24_Column_30,Angie,-111.974895,33.075375,41,121,595,...,294.336975,292.886627,294.316803,294.317780,295.507538,2.693689,1.641246,9,NaN,0
200447,7554,2020-02-24,treatment 2,MAC_Field_Scanner_Season_10_Range_24_Column_30,Angie,-111.974895,33.075375,62,149,604,...,296.036682,293.541809,295.777985,295.850830,298.052765,8.130352,2.851377,9,NaN,0


[nan]


In [11]:
# left join to get non nan values
# df_bool = flir.merge(dups.drop_duplicates(), on =  ['plant_name'], how = 'left', indicator = True);df_bool



In [7]:
flir = flir.dropna(axis=0)

is_dup_df = flir.duplicated(subset=['plant_name', 'date'],keep=False)
dups = flir[is_dup_df]
display(dups)


print(dups.plant_name.unique())

,index,date,treatment,plot,genotype,lon,lat,min_x,max_x,min_y,...,roi_temp,quartile_1,mean,median,quartile_3,variance,std_dev,labels,plant_name,double_lettuce


[]


In [9]:
# add flir

with sqlite3.connect('../po10.db') as con:
    flir_data.to_sql('flir', con=con, dtype={('plant_name', 'date'): 'STRING PRIMARY KEY'})

In [32]:
# add 3d

with sqlite3.connect('po.db') as con:
    scanner3d_data.to_sql('scanner3d', con=con, dtype={('plant_name', 'date'): 'STRING PRIMARY KEY'})

/home/travis/Downloads/yes/envs/general/lib/python3.9/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [20]:
con = sqlite3.connect('../po10.db')

q_result = pd.read_sql_query('select * from scanner3d limit 5', con)

q_result.head()

,plant_name,min_x,min_y,min_z,max_x,max_y,max_z,num_points,hull_volume,oriented_bounding_box,...,amplitude_silhouette_1,amplitude_silhouette_2,amplitude_heat_0,amplitude_heat_1,amplitude_heat_2,amplitude_persistence_image_0,amplitude_persistence_image_1,amplitude_persistence_image_2,Unnamed: 0,date
0,0,Aido_100,-0.687389,-0.595167,-0.150176,0.746004,0.606181,0.422280,89348.0,0.434232,...,0.004238,0.001013,0.000073,145.303819,25.233863,0.125093,680.452694,272.104394,5.423737,2020-02-04
1,1,Aido_123,-0.610331,-0.630662,-0.144634,0.685339,0.590249,0.388524,77074.0,0.431524,...,0.004376,0.001381,0.000120,132.971104,66.151870,1.032622,666.057662,479.197713,15.301617,2020-02-04
2,2,Aido_124,-0.633510,-0.469131,-0.132039,0.696690,0.568071,0.371267,80770.0,0.313059,...,0.004315,0.000979,0.000146,106.768368,20.405339,0.168299,531.643822,231.193092,4.792345,2020-02-04
3,3,Aido_170,-0.629430,-0.611257,-0.122805,0.673616,0.630074,0.410606,76509.0,0.424726,...,0.004288,0.001414,0.000109,249.416043,34.213248,1.050909,853.015157,352.659495,14.834217,2020-02-04
4,4,Aido_17,-0.719136,-0.638537,-0.172508,0.624592,0.620152,0.449815,97346.0,0.532415,...,0.004452,0.001043,0.000266,220.631958,78.159572,0.967256,910.722242,555.693687,25.633030,2020-02-04


In [16]:
rgb_result = pd.read_sql_query('select * from rgb where plant_name="Aido_100"', con)
rgb_result.head()


,index,date,plant_name,lon,lat,min_x,max_x,min_y,max_y,nw_lat,nw_lon,se_lat,se_lon,bounding_area_m2,double_plant,partial_scan,flags
0,349389,2019-12-21,Aido_100,-111.974951,33.074785,266.0,487.0,9235.0,9432.0,33.074785,-111.974951,33.074785,-111.97495,0.004169,0.0,0,"['no_dp', 'no_ps']"
1,349421,2019-12-30,Aido_100,-111.974951,33.074785,187.0,484.0,8996.0,9291.0,33.074784,-111.974951,33.074785,-111.97495,0.008833,0.0,0,"['no_dp', 'no_ps']"
2,349481,2020-01-02,Aido_100,-111.974951,33.074785,244.0,562.0,8915.0,9189.0,33.074785,-111.974951,33.074785,-111.97495,0.008755,0.0,0,"['no_dp', 'no_ps']"
3,349551,2020-01-05,Aido_100,-111.974951,33.074785,232.0,571.0,8939.0,9228.0,33.074785,-111.974951,33.074785,-111.97495,0.009831,0.0,0,"['no_dp', 'no_ps']"
4,349634,2020-01-07,Aido_100,-111.974951,33.074785,218.0,573.0,8929.0,9210.0,33.074785,-111.974951,33.074785,-111.97495,0.009996,0.0,0,"['no_dp', 'no_ps']"


In [21]:
flir_result = pd.read_sql_query('select * from flir where plant_name="NaN"', con)
flir_result.head()

,level_0,index,date,treatment,plot,genotype,lon,lat,min_x,max_x,...,roi_temp,quartile_1,mean,median,quartile_3,variance,std_dev,labels,plant_name,double_lettuce


In [57]:
# join on 3d since it has the smallest number


union_result = pd.read_sql_query('SELECT * FROM scanner3d LEFT OUTER JOIN rgb on scanner3d.plant_name = rgb.plant_name WHERE scanner3d.plant_name = "Aido_100"', con)
union_result.head()

,index,plant_name,min_x,min_y,min_z,max_x,max_y,max_z,num_points,hull_volume,...,nw_lat,nw_lon,se_lat,se_lon,bounding_area_m2,double_lettuce,Unnamed: 0.1,geometry,index_right,ID
0,0,Aido_100,-0.646141,-0.695795,-0.175155,0.642084,0.654622,0.432095,54945.0,0.546489,...,33.074785,-111.974951,33.074785,-111.97495,0.004169,0.0,None,None,None,None
1,0,Aido_100,-0.646141,-0.695795,-0.175155,0.642084,0.654622,0.432095,54945.0,0.546489,...,33.074784,-111.974951,33.074785,-111.97495,0.008833,0.0,None,None,None,None
2,0,Aido_100,-0.646141,-0.695795,-0.175155,0.642084,0.654622,0.432095,54945.0,0.546489,...,33.074785,-111.974951,33.074785,-111.97495,0.008755,0.0,None,None,None,None
3,0,Aido_100,-0.646141,-0.695795,-0.175155,0.642084,0.654622,0.432095,54945.0,0.546489,...,33.074785,-111.974951,33.074785,-111.97495,0.009831,0.0,None,None,None,None
4,0,Aido_100,-0.646141,-0.695795,-0.175155,0.642084,0.654622,0.432095,54945.0,0.546489,...,33.074785,-111.974951,33.074785,-111.97495,0.009996,0.0,None,None,None,None
